# A demo of the `LinearOperators.jl` library

Solve the Laplace equation `Δu = f` over a `M × N` grid using finite differences. The numerical scheme reads in the real space

In [ ]:
using LinearOperators

In [ ]:
const DIM = 2

In [ ]:
const T = Float64

In [ ]:
Lx, Ly = 1.0, 1.5

In [ ]:
M, N = 8, 16

In [ ]:
Δx, Δy = Lx / M, Ly / N

In [ ]:
nrows = ncols = M * N

In [ ]:
function mul_laplacian!(y, x, α, β)
    x_hat = fft(x)
    for m = 1:M
        k_x² = (2.0 / Δx * sin(π * m / M)) ^ 2
        for n = 1:N
            k_y² = (2.0 / Δy * sin(π * n / N))^2
            x_hat[m, n] *= k_x² + k_y²
        end
    end
    y .= α .* ifft(x_hat) .+ β .* y
end

In [ ]:
Δ = LinearOperator(ComplexF64, nrows, ncols, false, false, mul_laplacian!, nothing, nothing)

In [ ]:
Δx